<a href="https://colab.research.google.com/github/MhmdSafa7/Helmet_Detection_yolo/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
import shutil

base_path = '/content/drive/MyDrive/Colab Notebooks/Helmet_Data'
annotations_path = os.path.join(base_path, 'annotations')
images_path = os.path.join(base_path, 'images')

yolo_dataset_path = os.path.join(base_path, 'yolo_dataset')
os.makedirs(yolo_dataset_path, exist_ok=True)



In [ ]:
import os
import xml.etree.ElementTree as ET

CLASS_NAMES = ["helmet", "person", "head"]

# Work locally instead of directly in Drive
yolo_dataset_path = '/content/yolo_dataset'
annotations_path = '/content/drive/MyDrive/Colab Notebooks/Helmet_Data/annotations'
os.makedirs(yolo_dataset_path, exist_ok=True)
output_labels_dir = os.path.join(yolo_dataset_path, "labels")
os.makedirs(output_labels_dir, exist_ok=True)

def convert_voc_to_yolo(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find('size')
    if size is None:  # if file missing size info
        return
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    file_name = os.path.splitext(os.path.basename(xml_file))[0] + ".txt"
    with open(os.path.join(output_dir, file_name), "w") as out_file:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in CLASS_NAMES:
                continue
            cls_id = CLASS_NAMES.index(cls)

            xml_box = obj.find('bndbox')
            xmin = float(xml_box.find('xmin').text)
            ymin = float(xml_box.find('ymin').text)
            xmax = float(xml_box.find('xmax').text)
            ymax = float(xml_box.find('ymax').text)

            x_center = (xmin + xmax) / (2.0 * w)
            y_center = (ymin + ymax) / (2.0 * h)
            width = (xmax - xmin) / w
            height = (ymax - ymin) / h

            out_file.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Convert XMLs locally
for xml_file in os.listdir(annotations_path):
    if xml_file.endswith(".xml"):
        convert_voc_to_yolo(os.path.join(annotations_path, xml_file), output_labels_dir)

# Then copy the final dataset back to Drive safely
!cp -r /content/yolo_dataset /content/drive/MyDrive/Colab\ Notebooks/Helmet_Data/
print("✅ Conversion done and copied to Drive!")


In [ ]:
from sklearn.model_selection import train_test_split
import random

images = [f for f in os.listdir(images_path) if f.endswith('.jpg') or f.endswith('.png')]

train_imgs, temp_imgs = train_test_split(images, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.33, random_state=42)

splits = {'train': train_imgs, 'val': val_imgs, 'test': test_imgs}

for split, imgs in splits.items():
    img_dir = os.path.join(yolo_dataset_path, split, 'images')
    lbl_dir = os.path.join(yolo_dataset_path, split, 'labels')
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(lbl_dir, exist_ok=True)

    for img_name in imgs:
        shutil.copy(os.path.join(images_path, img_name), img_dir)
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        src_label = os.path.join(output_labels_dir, txt_name)
        if os.path.exists(src_label):
            shutil.copy(src_label, lbl_dir)


In [ ]:
yaml_content = f"""
train: {yolo_dataset_path}/train/images
val: {yolo_dataset_path}/val/images
test: {yolo_dataset_path}/test/images

nc: {len(CLASS_NAMES)}
names: {CLASS_NAMES}
"""

with open(os.path.join(yolo_dataset_path, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("✅ data.yaml created successfully!")
